In [1]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix,classification_report
!pip install six
from six import StringIO
from sklearn.tree import DecisionTreeClassifier 
from sklearn.tree import export_graphviz
#from sklearn.externals.six import StringIO  
from IPython.display import Image  

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, roc_auc_score, make_scorer

In [2]:
ad_data = pd.read_csv('train.csv')
ad_data.head()

,UserID,realtionship_status,industry,genre,targeted_sex,average_runtime(minutes_per_week),airtime,airlocation,ratings,expensive,money_back_guarantee,netgain
0,UI_9185,Married-civ-spouse,Pharma,Comedy,Male,40,Daytime,United-States,0.027465,Medium,No,0
1,UI_449,Never-married,Political,Comedy,Male,35,Morning,International,0.027465,Low,No,0
2,UI_9014,Never-married,Auto,Comedy,Male,40,Morning,United-States,0.027465,Low,No,0
3,UI_14987,Married-civ-spouse,Pharma,Infomercial,Male,40,Primetime,United-States,0.027465,Low,Yes,0
4,UI_25532,Married-civ-spouse,Other,Comedy,Female,50,Primetime,United-States,0.027465,Low,Yes,1


In [3]:
#Checking data shape

ad_data.shape

(19536, 12)

In [4]:
#Checking data info
ad_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19536 entries, 0 to 19535
Data columns (total 12 columns):
UserID                               19536 non-null object
realtionship_status                  19536 non-null object
industry                             19536 non-null object
genre                                19536 non-null object
targeted_sex                         19536 non-null object
average_runtime(minutes_per_week)    19536 non-null int64
airtime                              19536 non-null object
airlocation                          19536 non-null object
ratings                              19536 non-null float64
expensive                            19536 non-null object
money_back_guarantee                 19536 non-null object
netgain                              19536 non-null int64
dtypes: float64(1), int64(2), object(9)
memory usage: 1.8+ MB


In [5]:
#Desribe data for numerical data
ad_data.describe()

,average_runtime(minutes_per_week),ratings,netgain
count,19536.000000,19536.000000,19536.000000
mean,40.375665,0.038826,0.238022
std,12.483166,0.075971,0.425884
min,1.000000,0.000000,0.000000
25%,40.000000,0.027465,0.000000
50%,40.000000,0.027465,0.000000
75%,45.000000,0.027465,0.000000
max,99.000000,1.000000,1.000000


In [6]:
#Remove user id
ad_data.drop(columns ='UserID', inplace = True)
ad_data.head()

,realtionship_status,industry,genre,targeted_sex,average_runtime(minutes_per_week),airtime,airlocation,ratings,expensive,money_back_guarantee,netgain
0,Married-civ-spouse,Pharma,Comedy,Male,40,Daytime,United-States,0.027465,Medium,No,0
1,Never-married,Political,Comedy,Male,35,Morning,International,0.027465,Low,No,0
2,Never-married,Auto,Comedy,Male,40,Morning,United-States,0.027465,Low,No,0
3,Married-civ-spouse,Pharma,Infomercial,Male,40,Primetime,United-States,0.027465,Low,Yes,0
4,Married-civ-spouse,Other,Comedy,Female,50,Primetime,United-States,0.027465,Low,Yes,1


In [7]:
#Check NaN values

ad_data.isna().sum()

realtionship_status                  0
industry                             0
genre                                0
targeted_sex                         0
average_runtime(minutes_per_week)    0
airtime                              0
airlocation                          0
ratings                              0
expensive                            0
money_back_guarantee                 0
netgain                              0
dtype: int64

In [8]:
#Separate the numerical data and categorical columns
numerical_columns = list(ad_data._get_numeric_data().columns)
numerical_columns



['average_runtime(minutes_per_week)', 'ratings', 'netgain']

In [9]:
#Categorical data
categorical_columns = list(set(ad_data.columns) - set(ad_data._get_numeric_data().columns))
categorical_columns

['genre',
 'industry',
 'realtionship_status',
 'airtime',
 'targeted_sex',
 'expensive',
 'airlocation',
 'money_back_guarantee']

In [10]:
#ad_data.drop(['Trinadad&Tobago'],1,inplace=True)
#ad_data.shape

In [11]:
#converting categorical to numerical value by encoding them
for column in ['airtime','expensive','money_back_guarantee','genre','airlocation', 'industry', 'targeted_sex', 'realtionship_status']:
    ad_data = pd.concat((ad_data,pd.get_dummies(ad_data[column])),1)
    ad_data.drop(column,1,inplace = True)
ad_data

,average_runtime(minutes_per_week),ratings,netgain,Daytime,Morning,Primetime,High,Low,Medium,No,...,Political,Female,Male,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed
0,40,0.027465,0,1,0,0,0,0,1,1,...,0,0,1,0,0,1,0,0,0,0
1,35,0.027465,0,0,1,0,0,1,0,1,...,1,0,1,0,0,0,0,1,0,0
2,40,0.027465,0,0,1,0,0,1,0,1,...,0,0,1,0,0,0,0,1,0,0
3,40,0.027465,0,0,0,1,0,1,0,0,...,0,0,1,0,0,1,0,0,0,0
4,50,0.027465,1,0,0,1,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19531,40,0.027465,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,1,0,0
19532,15,0.027465,1,0,1,0,1,0,0,0,...,1,1,0,0,0,0,0,1,0,0
19533,55,0.027465,0,0,0,1,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
19534,38,0.027465,0,0,0,1,1,0,0,0,...,1,0,1,0,0,0,0,1,0,0


In [12]:
#Checking Class imbalance
print(ad_data.netgain.value_counts())
def class_imbalance(target):
    class_values = (target.value_counts()/target.value_counts().sum())*100
    return class_values

class_imbalance(ad_data['netgain'])

0    14886
1     4650
Name: netgain, dtype: int64


0    76.197789
1    23.802211
Name: netgain, dtype: float64

In [13]:
#Removing extra column from train data which is not in test data

#extra_cols = list(set(X.columns) - set(test.columns))
#extra_cols

ad_data.drop("Trinadad&Tobago",axis = 1,inplace=True)
ad_data.head()

,average_runtime(minutes_per_week),ratings,netgain,Daytime,Morning,Primetime,High,Low,Medium,No,...,Political,Female,Male,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed
0,40,0.027465,0,1,0,0,0,0,1,1,...,0,0,1,0,0,1,0,0,0,0
1,35,0.027465,0,0,1,0,0,1,0,1,...,1,0,1,0,0,0,0,1,0,0
2,40,0.027465,0,0,1,0,0,1,0,1,...,0,0,1,0,0,0,0,1,0,0
3,40,0.027465,0,0,0,1,0,1,0,0,...,0,0,1,0,0,1,0,0,0,0
4,50,0.027465,1,0,0,1,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0


In [14]:
#declaring features and target variable
X = ad_data.drop('netgain',1)
y = ad_data.netgain

In [15]:
#splitting data into train and test 
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 0, test_size = 0.4)

In [16]:
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,StandardScaler,RobustScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [17]:
# Applying Smote on train data for dealing with class imbalance

from imblearn.over_sampling import RandomOverSampler,SMOTE
smote = SMOTE(random_state=0)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [18]:
print(X_train.shape)
print(y_train.shape)

(17886, 71)
(17886,)


In [19]:
#Creating model using Decision Tree
DT_clf_gini = DecisionTreeClassifier(criterion='gini')
DT_clf_gini.fit(X_train, y_train)
y_pred_gini = DT_clf_gini.predict(X_test)

In [20]:
print(classification_report(y_test, y_pred_gini))

              precision    recall  f1-score   support

           0       0.90      0.79      0.84      5943
           1       0.51      0.71      0.59      1872

    accuracy                           0.77      7815
   macro avg       0.70      0.75      0.72      7815
weighted avg       0.80      0.77      0.78      7815



In [21]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred_gini, average='binary')

0.5947274352100089

In [22]:
#Creating random forest
clf_1 = RandomForestClassifier(min_samples_leaf=2, random_state=9)
clf_1.fit(X_train,y_train)

# predicting for min_samples 0f leaf as 2.
y_clf_1 = clf_1.predict(X_test)

#calculate accuracy and roc auc score for min leaf samples

accuracy_clf_1 = accuracy_score(y_test, y_clf_1)
auc_roc1 = roc_auc_score(y_test, y_clf_1)
print("Accuracy score of the model with min sample leaf 2 is: {}".format(accuracy_clf_1))
print("roc_auc_score of the model1 is: {}".format(auc_roc1))
print('F1score is - ', f1_score(y_test, y_clf_1, average='binary'))
print(classification_report(y_test, y_clf_1))

Accuracy score of the model with min sample leaf 2 is: 0.7689059500959693
roc_auc_score of the model1 is: 0.7832881929613378
F1score is -  0.6270136307311028
              precision    recall  f1-score   support

           0       0.93      0.76      0.83      5943
           1       0.51      0.81      0.63      1872

    accuracy                           0.77      7815
   macro avg       0.72      0.78      0.73      7815
weighted avg       0.83      0.77      0.78      7815



In [23]:
#Gradientboost

from sklearn.ensemble import GradientBoostingClassifier ,RandomForestClassifier ,GradientBoostingClassifier
model=GradientBoostingClassifier()
model.fit(X_train, y_train)
y_scores = model.predict(X_test)
print('F1 Score using GradientBoost - ', f1_score(y_test, y_scores, average='binary'))

print(classification_report(y_test, y_scores))

F1 Score using GradientBoost -  0.6359918200408997
              precision    recall  f1-score   support

           0       0.93      0.75      0.83      5943
           1       0.52      0.83      0.64      1872

    accuracy                           0.77      7815
   macro avg       0.72      0.79      0.74      7815
weighted avg       0.83      0.77      0.79      7815



# TEST DATA

In [24]:
test1=pd.read_csv("test.csv")
test1.describe()

,average_runtime(minutes_per_week),ratings
count,6512.000000,6512.000000
mean,40.049447,0.038385
std,12.466081,0.075497
min,1.000000,0.000000
25%,40.000000,0.027465
50%,40.000000,0.027465
75%,45.000000,0.027465
max,99.000000,1.000000


In [25]:
test1.shape

(6512, 11)

In [26]:
test=test1.copy()

In [27]:
#dropping column ID
test.drop("UserID",1,inplace=True)

In [28]:
#converting categorical to numerical value by encoding them
for column in ['airtime','expensive','money_back_guarantee','genre','airlocation', 'industry', 'targeted_sex', 'realtionship_status']:
    test = pd.concat((test,pd.get_dummies(test[column])),1)
    test.drop(column,1,inplace = True)
test

,average_runtime(minutes_per_week),ratings,Daytime,Morning,Primetime,High,Low,Medium,No,Yes,...,Political,Female,Male,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed
0,40,0.027465,0,0,1,1,0,0,0,1,...,0,1,0,0,0,0,0,1,0,0
1,40,0.027465,0,0,1,0,1,0,1,0,...,0,1,0,1,0,0,0,0,0,0
2,40,0.027465,0,0,1,1,0,0,1,0,...,0,0,1,0,0,1,0,0,0,0
3,30,0.027465,0,0,1,1,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0
4,70,0.027465,0,1,0,0,1,0,1,0,...,1,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6507,40,0.027465,0,0,1,0,1,0,0,1,...,1,0,1,0,0,0,0,1,0,0
6508,40,0.027465,0,0,1,0,1,0,0,1,...,0,1,0,1,0,0,0,0,0,0
6509,40,0.027465,0,0,1,1,0,0,0,1,...,0,0,1,0,0,1,0,0,0,0
6510,35,0.027465,0,0,1,0,1,0,0,1,...,0,1,0,1,0,0,0,0,0,0


In [29]:
#Prediction using Gradient Boost

Gb_Pred = model.predict(test)
Gb_Pred

array([0, 0, 1, ..., 1, 0, 0], dtype=int64)

In [30]:
data = {'netgain': Gb_Pred}
prediction = pd.DataFrame(data,columns=['netgain'],index=None)
prediction.tail()

,netgain
6507,0
6508,0
6509,1
6510,0
6511,0


In [32]:
prediction.to_csv('submissionKri.csv',index=False)

# The End